＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝  
ex5:  
特徴量を新たに加える
・onehot
・ある遺伝子集合の平均

後処理  
・近傍

とりあえず突っ込んだ
cv score : 0.016759827614837304 (control removed)
cv score : 0.01544

何もなし
cv score : 0.016659014838521283
cv score : 0.01535365993431876

random
cv score : 0.016784831050276998 
random(:400)
cv score : 0.016759801481980134

In [1]:
import warnings, random, os, sys, tqdm, time
sys.path.append("../")
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


from sklearn.decomposition import PCA
from sklearn.feature_selection import VarianceThreshold
from sklearn.metrics import log_loss
from sklearn.preprocessing import StandardScaler, MinMaxScaler, QuantileTransformer, RobustScaler
from sklearn.cluster import KMeans
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold


import torch
from torch import nn
import torch.optim as optim
import torch.nn.functional as F
from torch.optim.lr_scheduler import ReduceLROnPlateau

from pytorch_tabnet.tab_model import TabNetRegressor
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

pd.set_option("display.max_columns", 1200)
pd.set_option("display.max_rows", 1200)
%matplotlib inline

In [2]:
def metric(y_true, y_pred):
    res = []
    for i in range(0, y_true.shape[1]):
        y = y_true[:,i]
        pred = y_pred[:,i]
        res.append(log_loss(y, pred))
    return np.mean(res)

def seed_everything(seed_value):
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    os.environ['PYTHONHASHSEED'] = str(seed_value)
    
    if torch.cuda.is_available(): 
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = True
seed_everything(42)
        
    
def make_scaler(flag, seed):
    if flag == "quantile":
        return QuantileTransformer(n_quantiles=100,random_state=seed, output_distribution="normal")
    elif flag == "gauss":
        return GaussRankScaler()
    elif flag == "standard":
        return StandardScaler()
    elif flag == "minmax":
        return MinMaxScaler()
    elif flag == "robust":
        return RobustScaler()
    

seeds = [14, 15, 16, 17, 18, 19, 20]
SCALE = "quantile"

    

In [3]:
# g772, c100, 206クラス、402クラスの分類

train_df = pd.read_csv("../../../Data/Raw/train_features.csv")
test_df = pd.read_csv("../../../Data/Raw/test_features.csv")
#pub_test_df = pd.read_csv("../input/moapublictest/test_features.csv")
pub_test_df = pd.read_csv("../../../Data/Raw/test_features.csv")
drug_df = pd.read_csv("../../../Data/Raw/train_drug.csv")#

y = pd.read_csv("../../../Data/Raw/train_targets_scored.csv")
y_non = pd.read_csv("../../../Data/Raw/train_targets_nonscored.csv")
y_all = pd.concat([y, y_non.drop("sig_id", axis=1)], axis=1)
y = y.merge(drug_df, on='sig_id', how='left') #

GENES = [col for col in train_df.columns if col.startswith("g-")]
CELLS = [col for col in train_df.columns if col.startswith("c-")]
BIOS = GENES + CELLS


SCORED_MOAS = [col for col in y.columns if col != "sig_id" and col != "drug_id"]#
NONSCORED_MOAS = [col for col in y_non.columns if col != "sig_id"]
ALL_MOAS = SCORED_MOAS + NONSCORED_MOAS


TR_SIZE = train_df.shape[0]
TE_SIZE = test_df.shape[0]

train_nonvehicle_index = train_df[train_df["cp_type"] != "ctl_vehicle"].index
test_nonvehicle_index = test_df[test_df["cp_type"] != "ctl_vehicle"].index

train_df["time_dose"] = train_df["cp_time"].astype(str) + " * " + train_df["cp_dose"]
test_df["time_dose"] = test_df["cp_time"].astype(str) + " * " + test_df["cp_dose"]
pub_test_df["time_dose"] = pub_test_df["cp_time"].astype(str) + " * " + pub_test_df["cp_dose"]

# remove cp_type = ctl_vehicle
mask = train_df["cp_type"] != "ctl_vehicle"
train_df = train_df[mask].drop("cp_type", axis=1).reset_index(drop=True)
test_df = test_df[test_df["cp_type"] != "ctl_vehicle"].drop("cp_type", axis=1).reset_index(drop=True)
pub_test_df = pub_test_df[pub_test_df["cp_type"] != "ctl_vehicle"].drop("cp_type", axis=1).reset_index(drop=True)
y_nonv = y[mask].reset_index(drop=True)#

scored = y_nonv.copy()#
y_nonv.drop("drug_id", axis=1, inplace=True)#
y.drop("drug_id", axis=1, inplace=True)#

TR_NONV_SIZE = train_df.shape[0]
TE_NONV_SHAPE = test_df.shape[0]

In [4]:
from pytorch_tabnet.metrics import Metric
class LogitsLogLoss(Metric):
    """
    LogLoss with sigmoid applied
    """

    def __init__(self):
        self._name = "logits_ll"
        self._maximize = False

    def __call__(self, y_true, y_pred):
        """
        Compute LogLoss of predictions.

        Parameters
        ----------
        y_true: np.ndarray
            Target matrix or vector
        y_score: np.ndarray
            Score matrix or vector

        Returns
        -------
            float
            LogLoss of predictions vs targets.
        """
        logits = 1 / (1 + np.exp(-y_pred))
        aux = (1 - y_true) * np.log(1 - logits + 1e-15) + y_true * np.log(logits + 1e-15)
        return np.mean(-aux)
    
class LabelSmoothing(nn.Module):
    """
    NLL loss with label smoothing.
    """
    def __init__(self, smoothing=0.0, n_cls=2):
        """
        Constructor for the LabelSmoothing module.
        :param smoothing: label smoothing factor
        """
        super(LabelSmoothing, self).__init__()
        self.confidence = 1.0 - smoothing + smoothing / n_cls
        self.smoothing = smoothing / n_cls

    def forward(self, x, target):
        probs = torch.nn.functional.sigmoid(x,)
        # ylogy + (1-y)log(1-y)
        #with torch.no_grad():
        target1 = self.confidence * target + (1-target) * self.smoothing
        #print(target1.cpu())
        loss = -(torch.log(probs+1e-15) * target1 + (1-target1) * torch.log(1-probs+1e-15))
        #print(loss.cpu())
        #nll_loss = -logprobs.gather(dim=-1, index=target.unsqueeze(1))
        #nll_loss = nll_loss.squeeze(1)
        #smooth_loss = -logprobs.mean(dim=-1)
        #loss = self.confidence * nll_loss + self.smoothing * smooth_loss
        return loss.mean()


In [5]:
# prod
# 上位500こ
#prod_cols = [['g-145', 'g-201', 'g-208'], ['g-370', 'g-508', 'g-37'], ['g-38', 'g-392', 'g-707'], ['g-328', 'g-28', 'g-392'], ['g-441', 'g-157', 'g-392'], ['g-181', 'g-100', 'g-392'], ['g-67', 'g-760', 'g-50'], ['g-731', 'g-100', 'g-707'], ['g-478', 'g-468', 'g-310'], ['g-91', 'g-145', 'g-208'], ['g-106', 'g-744', 'g-91'], ['g-131', 'g-208', 'g-392'], ['g-144', 'g-123', 'g-86'], ['g-228', 'g-72', 'g-67'], ['g-31', 'g-328', 'g-460'], ['g-392', 'g-731', 'g-100'], ['g-732', 'g-744', 'g-707'], ['g-705', 'g-375', 'g-704'], ['g-508', 'g-50', 'g-411'], ['g-234', 'g-58', 'g-520'], ['g-503', 'g-761', 'g-50'], ['g-113', 'g-75', 'g-178'], ['g-50', 'g-508', 'g-113'], ['g-113', 'g-375', 'g-75'], ['g-576', 'g-452', 'g-392'], ['g-50', 'g-37', 'g-36'], ['g-707', 'g-133', 'g-392'], ['g-484', 'g-392', 'g-544'], ['g-508', 'g-67', 'g-370'], ['g-123', 'g-731', 'g-100'], ['g-298', 'g-477', 'g-644'], ['g-72', 'g-370', 'g-50'], ['g-67', 'g-178', 'g-113'], ['g-744', 'g-608', 'g-100'], ['g-91', 'g-100', 'g-707'], ['g-37', 'g-228', 'g-202'], ['g-37', 'g-300', 'g-370'], ['g-234', 'g-508', 'g-595'], ['g-596', 'g-744', 'g-707'], ['g-300', 'g-227', 'g-591'], ['g-135', 'g-392', 'g-512'], ['g-731', 'g-744', 'g-158'], ['g-69', 'g-707', 'g-100'], ['g-276', 'g-653', 'g-291'], ['g-624', 'g-615', 'g-189'], ['g-181', 'g-707', 'g-38'], ['g-72', 'g-75', 'g-508'], ['g-231', 'g-707', 'g-392'], ['g-508', 'g-37', 'g-72'], ['g-725', 'g-712', 'g-640'], ['g-67', 'g-644', 'g-113'], ['g-508', 'g-228', 'g-656'], ['g-185', 'g-37', 'g-672'], ['g-370', 'g-50', 'g-503'], ['g-201', 'g-745', 'g-599'], ['g-332', 'g-50', 'g-571'], ['g-50', 'g-37', 'g-59'], ['g-508', 'g-113', 'g-231'], ['g-707', 'g-158', 'g-100'], ['g-257', 'g-50', 'g-72']]
#prod_cols = [['g-712',  'g-208',  'g-38',  'g-100',  'g-123',  'g-328',  'g-744',  'g-248',  'g-460',  'g-731',  'g-417',  'g-349',  'g-131'], ['g-228',  'g-75',  'g-67',  'g-760',  'g-37',  'g-406',  'g-50',  'g-672',  'g-63',  'g-72',  'g-195'], ['g-100', 'g-157', 'g-178'],['g-183', 'g-300', 'g-767'],['g-50', 'g-37', 'g-489', 'g-257', 'g-332'],['g-270', 'g-135', 'g-231', 'g-158', 'g-478', 'g-146', 'g-491', 'g-392'],['g-745', 'g-635', 'g-235'], ['g-300', 'g-414', 'g-62', 'g-34'], ['g-91', 'g-392'], ['g-75', 'g-113'], ['g-599', 'g-261', 'g-38', 'g-146', 'g-392', 'g-512', 'g-744'], ['g-50', 'g-332', 'g-37', 'g-58', 'g-705'], ['g-157', 'g-178'],['g-759', 'g-100', 'g-167', 'g-75', 'g-431', 'g-189', 'g-522', 'g-91'],['g-202', 'g-385', 'g-769']            ]

prod_cols = [
    ['g-712',  'g-208',  'g-38',  'g-100',  'g-123',  'g-328',  'g-744',  'g-248',  'g-460',  'g-731',  'g-417',  'g-349',  'g-131'], ['g-228',  'g-75',  'g-67',  'g-760',  'g-37',  'g-406',  'g-50',  'g-672',  'g-63',  'g-72',  'g-195'], ['g-100', 'g-157', 'g-178'],['g-183', 'g-300', 'g-767'],['g-50', 'g-37', 'g-489', 'g-257', 'g-332'],['g-270', 'g-135', 'g-231', 'g-158', 'g-478', 'g-146', 'g-491', 'g-392'],['g-745', 'g-635', 'g-235'], ['g-300', 'g-414', 'g-62', 'g-34'], ['g-91', 'g-392'], ['g-75', 'g-113'], ['g-599', 'g-261', 'g-38', 'g-146', 'g-392', 'g-512', 'g-744'], ['g-50', 'g-332', 'g-37', 'g-58', 'g-705'], ['g-157', 'g-178'],['g-759', 'g-100', 'g-167', 'g-75', 'g-431', 'g-189', 'g-522', 'g-91'],['g-202', 'g-385', 'g-769'],
]

for cols in prod_cols:
    name = "prod-" + " * ".join(cols)
    train_df[name] = train_df[cols].mean(axis=1)
    test_df[name] = test_df[cols].mean(axis=1)
    pub_test_df[name] = pub_test_df[cols].mean(axis=1)

PRODS = [col for col in train_df.columns if col.startswith("prod-")]

In [6]:
#out fold preprocessing

#variance threshold

VAR_THRESHOLD = 0.8
drop_cols = []
temp = pd.concat([train_df, pub_test_df])
#for col in BIOS:
for col in BIOS+PRODS:
    if temp[col].var() <= VAR_THRESHOLD:
        drop_cols.append(col)

print("drop cols num : {}".format(len(drop_cols)))
train_df.drop(columns=drop_cols, inplace=True)
test_df.drop(columns=drop_cols, inplace=True)
pub_test_df.drop(columns=drop_cols, inplace=True)

GENES_ = [col for col in train_df.columns if col.startswith("g-")]
CELLS_ = [col for col in train_df.columns if col.startswith("c-")]
BIOS_ = GENES_ + CELLS_
PRODS = [col for col in train_df.columns if col.startswith("prod-")]
        
del temp

# onehot encode of categorical feature and drop
drop_cols = ["cp_time", "cp_dose", "time_dose"]
train_df = pd.concat([pd.get_dummies(train_df["time_dose"], prefix="onehot", drop_first=True), train_df.drop(drop_cols, axis=1) ], axis=1)
test_df = pd.concat([pd.get_dummies(test_df["time_dose"], prefix="onehot", drop_first=True), test_df.drop(drop_cols, axis=1) ], axis=1)
pub_test_df = pd.concat([pd.get_dummies(pub_test_df["time_dose"], prefix="onehot", drop_first=True), pub_test_df.drop(drop_cols, axis=1) ], axis=1)

# aggregation feature
print("agg")
for df in [train_df, pub_test_df, test_df]:
    df["sum-g"] = df[GENES_].sum(axis=1)
    df["mean-g"] = df[GENES_].mean(axis=1)
    df["std-g"] = df[GENES_].std(axis=1)
    df["kurt-g"] = df[GENES_].kurt(axis=1)
    df["skew-g"] = df[GENES_].skew(axis=1)
    df["sum-c"] = df[CELLS_].sum(axis=1)
    df["mean-c"] = df[CELLS_].mean(axis=1)
    df["std-c"] = df[CELLS_].std(axis=1)
    df["kurt-c"] = df[CELLS_].kurt(axis=1)
    df["skew-c"] = df[CELLS_].skew(axis=1)
    df["sum-gc"] = df[BIOS_].sum(axis=1)
    df["mean-gc"] = df[BIOS_].mean(axis=1)
    df["std-gc"] = df[BIOS_].std(axis=1)
    df["kurt-gc"] = df[BIOS_].kurt(axis=1)
    df["skew-gc"] = df[BIOS_].skew(axis=1)


drop cols num : 70
agg


In [7]:
X = train_df.drop("sig_id", axis=1)
y_nonv = y_nonv.drop("sig_id", axis=1).values
y = y.drop("sig_id", axis=1).values

In [13]:
MAX_EPOCH = 200
tabnet_params = dict(
    n_d = 24,
    n_a = 24,
    n_independent=1,
    n_shared=1,
    n_steps = 2,
    gamma = 1.3,
    lambda_sparse = 0,
    
    optimizer_fn = optim.Adam,
    optimizer_params = dict(lr = 2e-2, weight_decay = 1e-5),
    mask_type = "entmax",
    scheduler_fn = ReduceLROnPlateau,
    scheduler_params = dict(mode = "min", patience = 6, min_lr = 1e-5, factor = 0.9),
    #scheduler_fn = optim.lr_scheduler.OneCycleLR,
    #scheduler_params = dict(pct_start=0.1, div_factor=1e3, max_lr=1e-1, epochs=MAX_EPOCH),
    
    seed = 42,
    verbose = 10
)

train_preds = np.zeros((X.shape[0], y_nonv.shape[1]))
preds = np.zeros((test_df.shape[0], y_nonv.shape[1]))
imps = []
imp_cols = []
folds = []

for seed in seeds:
    seed_everything(seed)
    K = 5
    kf = MultilabelStratifiedKFold(n_splits=K, random_state=seed, shuffle=True)
    train_pred = np.zeros(train_preds.shape)
    
    ###############################################################################################
    # LOAD LIBRARIES
    targets = SCORED_MOAS.copy()

    # LOCATE DRUGS
    vc = scored["drug_id"].value_counts()
    vc1 = vc.loc[vc<=18].index.sort_values()
    vc2 = vc.loc[vc>18].index.sort_values()

    # STRATIFY DRUGS 18X OR LESS
    dct1 = {}; dct2 = {}
    skf = MultilabelStratifiedKFold(n_splits=K, shuffle=True, random_state=seed)
    tmp = scored.groupby('drug_id')[targets].mean().loc[vc1]
    for fold,(idxT,idxV) in enumerate( skf.split(tmp,tmp[targets])):
        dd = {k:fold for k in tmp.index[idxV].values} # drug id がどのフォールドに属すか格納
        dct1.update(dd)

    # STRATIFY DRUGS MORE THAN 18X
    skf = MultilabelStratifiedKFold(n_splits=K, shuffle=True, random_state=seed)
    tmp = scored.loc[scored["drug_id"].isin(vc2)].reset_index(drop=True)
    for fold,(idxT,idxV) in enumerate( skf.split(tmp,tmp[targets])):
        dd = {k:fold for k in tmp["sig_id"][idxV].values}
        dct2.update(dd)

    # ASSIGN K
    scored['fold'] = scored.drug_id.map(dct1)
    scored.loc[scored["fold"].isna(),'fold'] = scored.loc[scored["fold"].isna(),'sig_id'].map(dct2)
    scored["fold"] = scored["fold"].astype('int8')
    ###############################################################################################

    

    #for fold, (train_index, valid_index) in enumerate(kf.split(X, y_nonv)):
    for fold in range(K):
        train_index = scored[scored["fold"] != fold].index.to_list()
        valid_index = scored[scored["fold"] == fold].index.to_list()
        print("======================== fold {} ========================".format(fold+1))
        tabnet_params["seed"] = seed+(fold+1)
        folds.append(valid_index)
                
        # split data
        train_X = X.iloc[train_index]
        train_y = y_nonv[train_index]
        valid_X = X.iloc[valid_index]
        valid_y = y_nonv[valid_index]
        test_X = (test_df.drop("sig_id", axis=1))
        pub_test_X = (pub_test_df.drop("sig_id", axis=1))
        

            
    
        
        

        # scaler
        print(SCALE)
        scale_cols = BIOS_+PRODS
        scaler = make_scaler(SCALE, seed).fit(train_X.append(pub_test_X)[scale_cols])
        for df in [train_X, valid_X, test_X, pub_test_X]:
            df[scale_cols] = scaler.transform(df[scale_cols])

        
        print("PCA")
        #PCA
        n_decom_g = 80
        n_decom_c = 10
        decom_g_cols = [f"pca_g-{i}" for i in range(n_decom_g)]
        decom_c_cols = [f"pca_c-{i}" for i in range(n_decom_c)]
        
        pca_g = PCA(n_components = n_decom_g, random_state = seed).fit(train_X.append(pub_test_X)[GENES_])
        pca_c = PCA(n_components = n_decom_c, random_state = seed).fit(train_X.append(pub_test_X)[CELLS_])
        
        for df in [train_X, valid_X, test_X, pub_test_X]:
            df[decom_g_cols] = pca_g.transform(df[GENES_])
            df[decom_c_cols] = pca_c.transform(df[CELLS_])
            #df.drop(GENES_+CELLS_, axis=1, inplace=True)        

        # kmeans
        #print("kmeans clustering")
        #n_clusters = 5
        #kmeans = KMeans(n_clusters=n_clusters, random_state=seed).fit(train_X.append(pub_test_X)[GENES_])  
        #for df in [train_X, valid_X, test_X, pub_test_X]:
        #    df["kmeans"] = kmeans.predict(df[GENES_])

        #train_X = pd.concat([pd.get_dummies(train_X["kmeans"], prefix="kmeans", drop_first=True), train_X.drop("kmeans", axis=1) ], axis=1)
        #valid_X = pd.concat([pd.get_dummies(valid_X["kmeans"], prefix="kmeans", drop_first=True), valid_X.drop("kmeans", axis=1) ], axis=1)
        #test_X = pd.concat([pd.get_dummies(test_X["kmeans"], prefix="kmeans", drop_first=True), test_X.drop("kmeans", axis=1) ], axis=1)
        #pub_test_X = pd.concat([pd.get_dummies(pub_test_X["kmeans"], prefix="kmeans", drop_first=True), pub_test_X.drop("kmeans", axis=1) ], axis=1)
        
        
        

        

        print(f"feature num is {train_X.shape[1]}")
        imp_cols = train_X.columns
        # prepare data for training
        train_X = train_X.values
        valid_X = valid_X.values
        test_X = test_X.values
        
        
        # model training
        #tabnet_params["scheduler_params"]["steps_per_epoch"] = int(np.ceil(train_X.shape[0]/1024))
        model = TabNetRegressor(**tabnet_params)
        device = torch.device("cuda:1")
        model.device=device

        model.fit(
            X_train=train_X,
            y_train=train_y,
            eval_set=[(valid_X, valid_y)],
            eval_name = ["val"],
            eval_metric = ["logits_ll"],
            max_epochs=MAX_EPOCH,
            patience=20, 
            batch_size=1024, 
            virtual_batch_size=32,
            num_workers=1, 
            drop_last=False,
            #loss_fn=F.binary_cross_entropy_with_logits
            #loss_fn=LabelSmoothing(1e-5)
            loss_fn=LabelSmoothing(1e-5)
        )
        #model.load_model("../input/tabnetweights/tabnet_weights/{}_{}".format(seed, fold))

        val_preds = 1 / (1 + np.exp(-model.predict(valid_X)))
        train_pred[valid_index] +=  val_preds
        
        test_preds = 1 / (1 + np.exp(-model.predict(test_X)))
        preds += test_preds / (K*len(seeds))
        
        name = "{}_{}".format(seed, fold)
        model.save_model("tabnet_weights2/"+name)
        imps.append(model.feature_importances_)

        
    print("seed {} , cv score : {}".format(seed, metric(y_nonv, train_pred)))
    train_preds += train_pred/len(seeds)

print("cv score : {}".format(metric(y_nonv, train_preds)))

======================== fold 1 ========================
quantile
PCA
feature num is 927
Device used : cuda
epoch 0  | loss: 0.40907 | val_logits_ll: 0.06983 |  0:00:02s
epoch 10 | loss: 0.02021 | val_logits_ll: 0.02036 |  0:00:26s
epoch 20 | loss: 0.01827 | val_logits_ll: 0.01899 |  0:00:50s
epoch 30 | loss: 0.01742 | val_logits_ll: 0.01876 |  0:01:13s
epoch 40 | loss: 0.01646 | val_logits_ll: 0.01835 |  0:01:37s
epoch 50 | loss: 0.01596 | val_logits_ll: 0.01806 |  0:02:01s
epoch 60 | loss: 0.01538 | val_logits_ll: 0.01836 |  0:02:25s

Early stopping occured at epoch 68 with best_epoch = 48 and best_val_logits_ll = 0.01792
Best weights from best epoch are automatically used!
Successfully saved model at tabnet_weights2/14_0.zip
======================== fold 2 ========================
quantile


KeyboardInterrupt: 

In [ ]:
train_preds2 = np.zeros((TR_SIZE,  y.shape[1]))
train_preds2[train_nonvehicle_index] = train_preds


preds2 = np.zeros((TE_SIZE, y.shape[1]))
preds2[test_nonvehicle_index] = preds

print("cv score : {}".format(metric(y, train_preds2)))

In [ ]:
sub_df = pd.read_csv("../../../Data/Raw/sample_submission.csv")
#sub_df = pd.read_csv("../input/lish-moa/sample_submission.csv")
cols = [col for col in sub_df.columns if col != "sig_id"]
sub_df[cols] = preds2
sub_df.to_csv("submission.csv", index=False)

In [ ]:
train_sub = pd.read_csv("../../../Data/Raw/train_targets_scored.csv")
cols = [col for col in train_sub.columns if col != "sig_id"]
train_sub[cols] = train_preds2
train_sub.to_csv("train_preds.csv", index=False)

In [ ]:
importance = pd.DataFrame(np.array(imps).sum(axis=0), index=imp_cols, columns=['importance']).sort_values('importance', ascending=False)
importance[:500]*100

In [ ]:
importance[500:]

In [ ]:
product_cols = []
size = 500
for col in importance[:size].index:
        if col.startswith("prod-"):
            col = col.replace("prod-", "")
            product_cols.append(col.split(" * "))

In [ ]:
product_cols

In [ ]:
!zip tabnet_weights.zip tabnet_weights/*